In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm

class RegressionModel:
    def __init__(self, x, y, create_intercept=True, regression_type="logit"):
        self.x = x.copy()
        self.y = y.copy()
        self.create_intercept = create_intercept
        self.regression_type = regression_type
        self.results = {}

        # Add intercept if specified
        if self.create_intercept:
            self.x.insert(0, 'intercept', 1)

    def add_intercept(self):

        self.x['intercept'] = 1

    def ols_regression(self):
        """Performs Ordinary Least Squares (OLS) regression."""
        X = self.x.values
        Y = self.y.values
        beta = np.linalg.inv(X.T @ X) @ X.T @ Y
        residuals = Y - X @ beta
        variance = (residuals.T @ residuals) / (X.shape[0] - X.shape[1])
        standard_errors = np.sqrt(np.diag(variance * np.linalg.inv(X.T @ X)))

        # Populate results
        self.results = {}
        for i, col in enumerate(self.x.columns):
            coefficient = beta[i][0]
            standard_error = standard_errors[i]
            z_stat = coefficient / standard_error
            p_value = 2 * (1 - norm.cdf(np.abs(z_stat)))
            self.results[col] = {
                'coefficient': coefficient,
                'standard_error': standard_error,
                'z_stat': z_stat,
                'p_value': p_value
            }

    def log_likelihood(self, beta):
        """Calculate the log-likelihood for logistic regression."""
        X = self.x.values
        Y = self.y.values.ravel()
        linear_combination = X @ beta
        log_likelihood = np.sum(Y * linear_combination - np.log(1 + np.exp(linear_combination)))
        return log_likelihood

    def logistic_gradient(self, beta):
        """Calculate the gradient for logistic regression."""
        X = self.x.values
        Y = self.y.values.ravel()
        predictions = 1 / (1 + np.exp(-X @ beta))
        gradient = X.T @ (Y - predictions)
        return gradient

    def logistic_regression(self, learning_rate=0.001, max_iter=10000, tol=1e-6):
        """Performs Logistic Regression using gradient descent."""
        X = self.x.values
        n, k = X.shape
        beta = np.zeros(k)  # Initialize coefficients
        for i in range(max_iter):
            gradient = self.logistic_gradient(beta)
            beta_new = beta + learning_rate * gradient
            if np.linalg.norm(beta_new - beta, ord=1) < tol:
                beta = beta_new
                break
            beta = beta_new

        # Calculate standard errors, z-stats, and p-values
        predictions = 1 / (1 + np.exp(-X @ beta))
        W = np.diag(predictions * (1 - predictions))
        variance_matrix = np.linalg.inv(X.T @ W @ X)
        standard_errors = np.sqrt(np.diag(variance_matrix))

        # Populate results
        self.results = {}
        for i, col in enumerate(self.x.columns):
            coefficient = beta[i]
            standard_error = standard_errors[i]
            z_stat = coefficient / standard_error
            p_value = 2 * (1 - norm.cdf(np.abs(z_stat)))
            self.results[col] = {
                'coefficient': coefficient,
                'standard_error': standard_error,
                'z_stat': z_stat,
                'p_value': p_value
            }

    def fit_model(self):
        """Determines which regression method to use and fits the model."""
        if self.regression_type == "ols":
            self.ols_regression()
        elif self.regression_type == "logit":
            self.logistic_regression()
        else:
            raise ValueError("Unsupported regression type. Choose 'ols' or 'logit'.")

    def summary(self):
        """Prints a summary of the regression results."""
        if self.regression_type == "ols":
            print(f"{'Variable':<12}{'Coefficient':<15}{'Std. Error':<15}{'z-Statistic':<15}{'P-Value':<15}")
            for variable, metrics in self.results.items():
                print(f"{variable:<12}{metrics['coefficient']:<15.6f}{metrics['standard_error']:<15.6f}"
                      f"{metrics['z_stat']:<15.6f}{metrics['p_value']:<15.6f}")
        elif self.regression_type == "logit":
            print(f"{'Variable':<12}{'Log Odds':<15}{'Std. Error':<15}{'z-Statistic':<15}{'P-Value':<15}")
            for variable, metrics in self.results.items():
                print(f"{variable:<12}{metrics['coefficient']:<15.6f}{metrics['standard_error']:<15.6f}"
                      f"{metrics['z_stat']:<15.6f}{metrics['p_value']:<15.6f}")
